GGH Cipher Demo
======================================

In this notebook, we present a basic walkthrough of the cryptosystem invented by Goldreich, Goldwasser and Halevi in 1997 [1].

__Note for using Jupyter:__ Use $\texttt{SHIFT + ENTER}$ to execute the code snippets!

We begin with some theoretical discussion, in particular, let's see the hardness assumption on which we can base the cipher.

Recall the __$\gamma(n)$-Closest Vector Problem (CVP)__:

Given a security parameter $\lambda = 1^n$, approximation factor $\gamma(n)$, a PPT adversary $\mathcal{A}$ and PPT algorithm $\texttt{GenCVP}$, the CVP experiment is defined as follows:

1. Generate $(\mathbf{u}, P, S) \leftarrow \texttt{GenCVP}(\lambda)$. Here $P, S \in \text{GL}_n(\mathbb{Q})$ such that $\mathcal{L}(P) = \mathcal{L}(S)$ and $\mathbf{u} \in \text{Span}(P)$.
2. Run $\mathbf{v} \leftarrow \mathcal{A}(\mathbf{u},P)$, where $\mathbf{v} \in \mathcal{L}(S)$
3. The adversary succeeds iff $||\mathbf{v} - \mathbf{u}|| \leq \gamma(n)\cdot ||\mathbf{v} - \mathbf{w}||$ for all $\mathbf{w} \in \mathcal{L}(P)$.

The hardness assumption of the above then is as follows:

$\gamma(n)$-CVP is hard relative to $\texttt{GenCVP}$ if every adversary succeeds with negligible advantage:

$$
    \texttt{Adv}^{\text{cvp}}_{\mathcal{A}, \texttt{GenCVP}}(n) = 
    2 \cdot \mathbb{P}(\texttt{Expr}^{\text{cvp}}_{\mathcal{A}, \texttt{GenCVP}}(n) - 1 \leq \text{negl}(n).
$$

Informally, the CVP assumes that we can generate a lattice basis in PPT, such that given the basis and some vector from the ambient space, it is hard to find the closest lattice vector to the point in the ambient space.

The trapdoor we can use here follows from a result by Laszlo Babai, who showed given that the vectors of a lattice basis are sufficiently orthogonal to each other (formally, the basis is Lovasz reduced) we always recover the $\gamma(n)$-closest vector, where

$$
    \gamma(n) = 1 + 2n\cdot\left(\frac{9}{2}\right)^{\frac{n}{2}}
$$
with $n$ being the dimension of the lattice.

Hence, we can exploit this by setting the public key as a "bad" basis, i.e. one with almost parallel vectors in it and setting the private key as a "good" basis, i.e. one with almost orthogonal vectors in it.

In [1]:
from sage.modules.free_module_integer import IntegerLattice
load('ggh.sage')

Key Generation
---------------------------------------------------------------------

Since our public and private bases have to generate the same lattice, it makes sense to find a "good" basis first, and then come up with a "bad" one based on it.

Hence, for the private basis, given a security parameter $\lambda = 1^n$ we pick $n$ random vectors from an n-dimensional hypercube with side lengths $2n$ (in $\mathbb{Z}^n$). We check how orthogonal this basis is by calculating the basis' Hadamard ratio and if it is above a certain threshold, we accept it by setting it as our secret key.

Then, we calculate the Hermite Normal Form (HNF) of our good basis, and set it as our public key. The reason why we do this is because the HNF of any lattice basis always exists, it can be calculated in polynomial time and in a sense it is the "worst" possible basis (see [2] for a rigorous explanation).

Below, we perform exactly this, by generating two 20-dimensional bases:

In [2]:
pk, sk = key_gen(20);

Hadamard ratio of PubK: 3.2652e-29
Hadamard ratio of PrivK: 0.71773


Encryption
---------------------------------------------------------------------

Given a public key $pk = P$ and some message $m$, we first convert $m$ to a (list of) $n$-dimensional vector $\mathbf{m}$. Then, we generate some small perturbation, by picking an $n$-dimensional vector $\mathbf{r}$ with each of its entries chosen uniformly such that $|r_i| \leq \delta$ for some small $\delta$.
Finally, we calculate and output the ciphertext

$$
    \mathbf{c} = \mathbf{m} \times P + \mathbf{r}.
$$

In our implementation, we chose $\delta = 4$, and the way we perform the message conversion, is by taking a string and converting it to a list of the ASCII codes of its characters. 

In [3]:
m = "You can't get to the moon by climbing successively taller trees."

c = encrypt(pk, m)

perturbation: [-2 -4 -1 -4  1 -4  0 -2  0  0  2  1 -4 -1  3  0  0 -3 -1  3]
perturbation: [ 1  2 -1  3 -4 -4  3 -4  4  4 -1  3 -1  3  2 -1  0  1  0  2]
perturbation: [ 4 -4 -3  4  1 -4  2  4 -3 -3 -1  3  3  1 -1 -2 -2 -3 -1  2]
perturbation: [ 4 -4 -3  4 -2 -1 -1  0 -2 -1  0 -2 -4 -1 -4  4  2  4 -1 -1]


Decryption
---------------------------------------------------------------------

Given some secret key $sk = (P, S)$ and some ciphertext $\mathbf{c}$, we use Babai's algorithm to recover the closest lattice vector. Thus, we calculate and output

$$
    \mathbf{m}' = \lfloor \mathbf{c} \times S^{-1} \rceil \times S \times P^{-1}.
$$

Correctness
----------------------------------------------------------------------

To see the correctness of the above procedure, given $pk = P, sk = (P, S)$ and some (converted) message $\mathbf{m}$, consider

$$
    \mathbf{m'} = \texttt{Dec}_{sk}(\texttt{Enc}_{pk}(\mathbf{m}))\\
    = \lfloor (\mathbf{m} \times P + \mathbf{r})\times S^{-1} \rceil \times S \times P^{-1}.
$$

Focus on the rounding operation for a moment. We know that $\mathbf{m} \times P$ is in $\mathcal{L}(P) = \mathcal{L}(S)$, so $\mathbf{m} \times P \times S^{-1}$ will be a vector with integer coefficients. Then, the rounding will give us the closest lattice vector as long as the entries of $\mathbf{r}\times S^{-1}$ are in the range $[-1/2, 1/2)$ (this is why we need to pick a sufficiently small $\delta$). Assuming it is the case, after the rounding operation we have

$$
    \lfloor (\mathbf{m} \times P + \mathbf{r})\times S^{-1} \rceil =
    \mathbf{m} \times P \times S^{-1}
$$

Substituting this into the above equation, we get

$$
    \mathbf{m'} = \mathbf{m} \times P \times S^{-1} \times S \times P^{-1} = \mathbf{m},
$$
thus the scheme is correct.

To see this working in practice:

In [4]:
decrypt(sk, c)

"You can't get to the moon by climbing successively taller trees."

Visual representation of the cipher
---------------------------------------------------------------------

Since we are dealing with geometric objects, we can get a nice visual intuition about how the cipher works, at least in 2 dimensions:

![alt GGH visual](GGH.png)

Here the green arrows represent the private basis $S$, the red arrows the public basis $P$.

Then, assume our message is given by the coordinates of the yellow vertex, the perturbation is the yellow vector and the blue vertex is the ciphertext we transmit.

Then, using the rounding operation, if we try to use the public basis, we will decode to the red vertex as it is the closest point in the public basis' fundamental domain! However, if we use the private basis, we correctly decode to the yellow vertex, recovering the message.

From this picture, we also see that since we are relying on an approximation factor of $1$, the perturbation vector cannot be too long.

Security Considerations
--------------------------------------------------------------

We note that the above cipher is secure, as long as it is hard to transform the public basis $P$ into a good basis $B$. The best currently known algorithm to do this is the so-called LLL algorithm invented by Lenstra, Lenstra and Lovasz, which runs in exponential time in the dimension of the lattice. However, we still have to make sure that our lattice of high enough dimension that the exponential nature starts to matter. This means that the recommended lattice dimension is $n > 500$. 

To demonstrate this issue, we will break scheme as follows:

In [5]:
L = IntegerLattice(pk[0], lll_reduce=False)

B = L.LLL()

broken_sk = (B, pk[0].inverse())

decrypt(broken_sk, c)

"You can't get to the moon by climbing successively taller trees."

References
--------------------------

[1] Oded Goldreich, Shafi Goldwasser, and Shai Halevi. “Public-key cryptosystems from lattice
reduction problems”. In: Annual International Cryptology Conference. Springer. 1997, pp. 112–
131.

[2] Daniele Micciancio. “Improving lattice based cryptosystems using the Hermite normal form”.
In: Cryptography and lattices. Springer, 2001, pp. 126–145.